In [4]:
#pip install opencv-python
#pip install tensorflow
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import zipfile
import cv2
import tensorflow as tf
from tensorflow.python.keras import Sequential
from tensorflow.keras import layers, optimizers
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import (Input, Add, Dense, Activation,
ZeroPadding2D, BatchNormalization, Flatten, Conv2D, 
AveragePooling2D, MaxPooling2D, Dropout)
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import (ReduceLROnPlateau,
EarlyStopping, ModelCheckpoint, LearningRateScheduler)
from IPython.display import display
from tensorflow.keras import backend as K
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
import plotly.express as px
import plotly.graph_objects as go

In [101]:
df = pd.read_csv('DataSet/sales_data_sample.csv', sep=',', decimal='.', encoding = "ISO-8859-1")
df.groupby('COUNTRY').size()
#df["ORDERDATE"] = pd.to_datetime(df["ORDERDATE"])
df = df.drop(["PHONE", "ADDRESSLINE1", "ADDRESSLINE2", "POSTALCODE", "CONTACTLASTNAME", "CONTACTFIRSTNAME"], axis=1)
df

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,CITY,STATE,COUNTRY,TERRITORY,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,Motorcycles,95,S10_1678,Land of Toys Inc.,NYC,NY,USA,NaN,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,Motorcycles,95,S10_1678,Reims Collectables,Reims,NaN,France,EMEA,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,Motorcycles,95,S10_1678,Lyon Souveniers,Paris,NaN,France,EMEA,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,Motorcycles,95,S10_1678,Toys4GrownUps.com,Pasadena,CA,USA,NaN,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,Motorcycles,95,S10_1678,Corporate Gift Ideas Co.,San Francisco,CA,USA,NaN,Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2818,10350,20,100.00,15,2244.40,12/2/2004 0:00,Shipped,4,12,2004,Ships,54,S72_3212,Euro Shopping Channel,Madrid,NaN,Spain,EMEA,Small
2819,10373,29,100.00,1,3978.51,1/31/2005 0:00,Shipped,1,1,2005,Ships,54,S72_3212,"Oulu Toy Supplies, Inc.",Oulu,NaN,Finland,EMEA,Medium
2820,10386,43,100.00,4,5417.57,3/1/2005 0:00,Resolved,1,3,2005,Ships,54,S72_3212,Euro Shopping Channel,Madrid,NaN,Spain,EMEA,Medium
2821,10397,34,62.24,1,2116.16,3/28/2005 0:00,Shipped,1,3,2005,Ships,54,S72_3212,Alpha Cognac,Toulouse,NaN,France,EMEA,Small


In [ ]:
#per la correcte importacio del csv es necessari utilitzar el encoding ISO-8859-1 o unicode_escape.
#de moment treballarem amn ISO-8859-1
df.info()

In [ ]:
#Precio pedidos por pais
x = df["SALES"]
y = df["MONTH_ID"]
dataX = np.array(x)
dataY = np.array(y)
data = np.hstack(dataX, dataY, axis=1)
'''
for row in dataX:
    print(row)
'''
print(dataX)
#plt.scatter(y, x)

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmeans = KMeans(n_clusters=3).fit(data)
centroids = kmeans.cluster_centers_

In [ ]:
plt.scatter(data[:,0], data[:,1])

In [ ]:
df.groupby("PRODUCTLINE").size()

In [ ]:
#2512 NaN en ADDRESSLINE2
#1486 NaN en STATE
#76   NaN en POSTALCODE
#1074 NaN en TERRITORY
for column in df.columns:
    print(column)
    print(df[df[column].isna()])

In [ ]:
prices = df["PRICEEACH"]
msrp = df["MSRP"]
print(prices)
print(msrp)

In [ ]:
for code in df.groupby("PRODUCTCODE").groups:
    print(code, end=': ')
    print(len(df[df["PRODUCTCODE"] == code]))

In [ ]:
muestra = df[df["PRODUCTCODE"] == "S10_1678"]
for col in muestra.columns:
    print(muestra[col])

In [ ]:
df[.groupby("PRODUCTCODE").size()]

In [108]:
#MINIRETO 4.1
print(len(df['STATE'].value_counts().index))
#MINIRETO 4.2

#Ara per ara no creiem necessari balancejar cap camp per l'ús que els hi volem donar. No obstant No ens tanquem les portes a haver de fer-ho mes endavant si s'escau.
#Camps que creiem que poden necessitar balanceix: COUNTRY, ORDERLINENUMBER, ORDERNUMBER

#MINIRETO 4.3 i 4.4 Función para agregar variables ficticias para reemplazar variables categóricas
def modificarVariables(variables):
    for variable in variables:
        print(variable)
        dicc = dict(zip(df.groupby(variable).groups, range(0, len(df.groupby(variable).groups))))
        df.replace({variable : dicc}, inplace=True)
        
modificarVariables(["STATUS", "PRODUCTLINE", "DEALSIZE", "COUNTRY"])

#MINIRETO 5
orderLine_dummy = pd.get_dummies(df["ORDERLINENUMBER"], prefix= "ORDERLINENUMBER")
df = pd.concat([df, orderLine_dummy], axis = 1)
df

16
STATUS
PRODUCTLINE
DEALSIZE
COUNTRY


,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,CITY,STATE,COUNTRY,TERRITORY,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,5,1,2,2003,1,95,S10_1678,Land of Toys Inc.,NYC,NY,18,NaN,2
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,5,2,5,2003,1,95,S10_1678,Reims Collectables,Reims,NaN,6,EMEA,2
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,5,3,7,2003,1,95,S10_1678,Lyon Souveniers,Paris,NaN,6,EMEA,1
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,5,3,8,2003,1,95,S10_1678,Toys4GrownUps.com,Pasadena,CA,18,NaN,1
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,5,4,10,2003,1,95,S10_1678,Corporate Gift Ideas Co.,San Francisco,CA,18,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2818,10350,20,100.00,15,2244.40,12/2/2004 0:00,5,4,12,2004,3,54,S72_3212,Euro Shopping Channel,Madrid,NaN,14,EMEA,2
2819,10373,29,100.00,1,3978.51,1/31/2005 0:00,5,1,1,2005,3,54,S72_3212,"Oulu Toy Supplies, Inc.",Oulu,NaN,5,EMEA,1
2820,10386,43,100.00,4,5417.57,3/1/2005 0:00,4,1,3,2005,3,54,S72_3212,Euro Shopping Channel,Madrid,NaN,14,EMEA,1
2821,10397,34,62.24,1,2116.16,3/28/2005 0:00,5,1,3,2005,3,54,S72_3212,Alpha Cognac,Toulouse,NaN,6,EMEA,2


In [71]:
import plotly.express as px
x = 'MSRP'
fig = plt.Figure(figsize = (12, 6))
fig = px.bar(x = df[x].value_counts().index, y = df[x].value_counts(), color =
df[x].value_counts().index, height = 600)
fig.show()

In [39]:
# Función para agregar variables ficticias para reemplazar variables categóricas
x = ["STATUS", "ASAS", "ASAS"]
df.groupby("STATUS").groups['Cancelled']


Int64Index([ 111,  117,  173,  303,  354,  406,  460,  539,  557,  564,  612,
             638,  868,  940,  946,  947, 1000, 1037, 1077, 1177, 1220, 1227,
            1301, 1525, 1595, 1602, 1749, 1756, 1804, 1824, 1831, 1880, 1899,
            1906, 1949, 1955, 1956, 1982, 2009, 2312, 2363, 2462, 2469, 2494,
            2521, 2539, 2546, 2573, 2593, 2600, 2676, 2696, 2703, 2722, 2729,
            2755, 2775, 2782, 2802, 2809],
           dtype='int64')